In [2]:
# 라이브러리 불러오기
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select

import time

In [3]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import urllib.request as req

from time import sleep

In [3]:
# 크롬 드라이버 잘 작동하는지 확인
driver = webdriver.Chrome() # 경로 지정해서 넣기
driver.get('https://esg.krx.co.kr/contents/02/02020000/ESG02020000.jsp')
time.sleep(3)
driver.quit()
# 3초 후 종료가 되는 것을 보면 크롬 드라이버는 잘 작동한다는 것을 알 수 있다

In [4]:
#2023년 esg등급 표 스크랩 함수
def esg_2023_scraper():
    driver = webdriver.Chrome()
    driver.get('https://esg.krx.co.kr/contents/02/02020000/ESG02020000.jsp')
    time.sleep(10) # 웹사이트가 켜지는 시간동안 잠시 멈춰 오류가 발생하는 것을 방지한다
    
    data_df_2023 = pd.DataFrame(columns=("회사명", "KCGS_종합등급", "KCGS_환경", "KCGS_사회", "KCGS_지배구조", "한국ESG연구소", "Moody's", "MSCI", "S&P", "연도"))
    
    year = 2023 # 매번 크롤링할 필요가 없으니 미리 선언하고 추가할 때 붙여주기만 하면 된다
    index = 0;  LIMIT = 794

    while index < LIMIT: # 총 기업이 794개가 있기 때문에 794로 제한을 둔다
        
        # 한 페이지 속 기업들(rows)에 대해(마지막 페이지를 제외하고 기본적으로 10개의 기업으로 구성)
        for i in range(1, 11):
            time.sleep(3)
            print(index + 1, end=' ') # 코드가 잘 돌아가는지 확인하기 위한 출력
    
            row = driver.find_element(By.CSS_SELECTOR, f"tbody > tr:nth-child({i})")
    
            row_data = row.text.replace('\n', ' ').split() # 빈칸을 기준으로 나누어 저장
            # 기업명, KCGS_종합 등급, KCGS_환경 등급, KCGS_사회 등급, KCGS_지배구조 등급, 한국ESG연구소 등급, 서스틴베스트 등급, Moody's 등급, MSCI 등급, S&P 등급
            print(row_data)
            if len(row_data) == 10: # 평범한 케이스
                company_name, kcgs_1, kcgs_2, kcgs_3, kcgs_4, kesg, sustin, moodys, msci, sp = row_data
        
            elif len(row_data) > 10: # 기업의 이름이 공백을 포함한 문자열인 경우 (ex. 'CJ' 'CGV'로 나뉘는 경우 'CJ CGV'로 합쳐야 한다)
                company_name = ' '.join(row_data[:len(row_data)-10+1])
                row_data = row_data[len(row_data)-10+1:] # 앞부분을 제외하여 슬라이싱
                kcgs_1, kcgs_2, kcgs_3, kcgs_4, kesg, sustin, moodys, msci, sp = row_data # 나머지 부분들 할당
                
            else: # 예상하지 못한 오류 상황
                company_name, kcgs_1, kcgs_2, kcgs_3, kcgs_4, kesg, sustin, moodys, msci, sp = ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-'] # 공백으로 처리
            
            # 기업별 추출한 정보를 dataframe의 한 행으로 저장
            data_df_2023.loc[len(data_df_2023)] = [company_name, kcgs_1, kcgs_2, kcgs_3, kcgs_4, kesg, moodys, msci, sp, year]

            # index를 증가시키기 -> while이 반복되며 index가 LIMIT이 되면 종료
            index += 1
            if index == LIMIT:
                break
        
        # for문을 다 돌아서 10개의 기업 정보를 크롤링하고 나면 다음 페이지를 클릭해야 한다
        next_page = driver.find_element(By.CSS_SELECTOR, "ul > li.next > a")
        next_page.click()
        time.sleep(5) # 다음 페이지가 로딩될 때까지 일정 시간 기다려서 오류를 방지
        
    # while문 종료
    data_df_2023.to_csv('esg_2023.csv')
    return data_df_2023

In [6]:
#2022년 esg등급 표 스크랩 함수
def esg_2022_scraper():
    driver = webdriver.Chrome()
    driver.get('https://esg.krx.co.kr/contents/02/02020000/ESG02020000.jsp')
    time.sleep(10)
    
    # 연도를 2022년으로 바꾸고 정보 조회
    select = Select(driver.find_element(By.CSS_SELECTOR, "#sch_yyc4ca4238a0b923820dcc509a6f75849b"))
    select.select_by_value('2022')
    driver.find_element(By.CSS_SELECTOR, "#btnidc4ca4238a0b923820dcc509a6f75849b").click()
    time.sleep(10)
    
    data_df_2022 = pd.DataFrame(columns=("회사명", "KCGS_종합등급", "KCGS_환경", "KCGS_사회", "KCGS_지배구조", "한국ESG연구소", "Moody's", "MSCI", "S&P", "연도"))
    
    year = 2022 # 매번 크롤링할 필요가 없으니 미리 선언하고 추가할 때 붙여주기만 하면 된다
    index = 0;  LIMIT = 794

    while index < LIMIT: # 총 기업이 794개가 있기 때문에 794로 제한을 둡니다.
        
        # 한 페이지 속 기업들(rows)에 대해(마지막 페이지를 제외하고 기본적으로 10개의 기업으로 구성)
        for i in range(1, 11):
            time.sleep(3)
            print(index + 1, end=' ') # 코드가 잘 돌아가는지 확인하기 위한 출력
    
            row = driver.find_element(By.CSS_SELECTOR, f"tbody > tr:nth-child({i})")
    
            row_data = row.text.replace('\n', ' ').split() # 빈칸을 기준으로 나누어 저장
            # 기업명, KCGS_종합 등급, KCGS_환경 등급, KCGS_사회 등급, KCGS_지배구조 등급, 한국ESG연구소 등급, 서스틴베스트 등급, Moody's 등급, MSCI 등급, S&P 등급
            print(row_data)
            if len(row_data) == 10: # 평범한 케이스
                company_name, kcgs_1, kcgs_2, kcgs_3, kcgs_4, kesg, sustin, moodys, msci, sp = row_data
        
            elif len(row_data) > 10: # 기업의 이름이 공백을 포함한 문자열인 경우 (ex. 'CJ' 'CGV'로 나뉘는 경우 'CJ CGV'로 합쳐야 한다)
                company_name = ' '.join(row_data[:len(row_data)-10+1])
                row_data = row_data[len(row_data)-10+1:] # 앞부분을 제외하여 슬라이싱
                kcgs_1, kcgs_2, kcgs_3, kcgs_4, kesg, sustin, moodys, msci, sp = row_data # 나머지 부분들 할당
                
            else: # 예상하지 못한 오류 상황
                company_name, kcgs_1, kcgs_2, kcgs_3, kcgs_4, kesg, sustin, moodys, msci, sp = ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-'] # 공백으로 처리
            
            # 기업별 추출한 정보를 dataframe의 한 행으로 저장
            data_df_2022.loc[len(data_df_2022)] = [company_name, kcgs_1, kcgs_2, kcgs_3, kcgs_4, kesg, moodys, msci, sp, year]

            # index를 증가시키기 -> while이 반복되며 index가 LIMIT이 되면 종료
            index += 1
            if index == LIMIT:
                break
        
        # for문을 다 돌아서 10개의 기업 정보를 크롤링하고 나면 다음 페이지를 클릭해야 한다
        next_page = driver.find_element(By.CSS_SELECTOR, "ul > li.next > a")
        next_page.click()
        time.sleep(5) # 다음 페이지가 로딩될 때까지 일정 시간 기다려서 오류를 방지
        
    # while문 종료
    data_df_2022.to_csv('esg_2022.csv')
    return data_df_2022
    

In [7]:
#2021년 esg등급 표 스크랩 함수
def esg_2021_scraper():
    driver = webdriver.Chrome()
    driver.get('https://esg.krx.co.kr/contents/02/02020000/ESG02020000.jsp')
    time.sleep(10)
    
    # 연도를 2021년으로 바꾸고 정보 조회
    select = Select(driver.find_element(By.CSS_SELECTOR, "#sch_yyc4ca4238a0b923820dcc509a6f75849b"))
    select.select_by_value('2021')
    driver.find_element(By.CSS_SELECTOR, "#btnidc4ca4238a0b923820dcc509a6f75849b").click()
    time.sleep(10)
    
    data_df_2021 = pd.DataFrame(columns=("회사명", "KCGS_종합등급", "KCGS_환경", "KCGS_사회", "KCGS_지배구조", "한국ESG연구소", "Moody's", "MSCI", "S&P", "연도"))
    
    year = 2021 # 매번 크롤링할 필요가 없으니 미리 선언하고 추가할 때 붙여주기만 하면 된다
    index = 0;  LIMIT = 794

    while index < LIMIT: # 총 기업이 794개가 있기 때문에 794로 제한을 둡니다.
        
        # 한 페이지 속 기업들(rows)에 대해(마지막 페이지를 제외하고 기본적으로 10개의 기업으로 구성)
        for i in range(1, 11):
            time.sleep(3)
            print(index + 1, end=' ') # 코드가 잘 돌아가는지 확인하기 위한 출력
    
            row = driver.find_element(By.CSS_SELECTOR, f"tbody > tr:nth-child({i})")
    
            row_data = row.text.replace('\n', ' ').split() # 빈칸을 기준으로 나누어 저장
            # 기업명, KCGS_종합 등급, KCGS_환경 등급, KCGS_사회 등급, KCGS_지배구조 등급, 한국ESG연구소 등급, 서스틴베스트 등급, Moody's 등급, MSCI 등급, S&P 등급
            print(row_data)
            if len(row_data) == 10: # 평범한 케이스
                company_name, kcgs_1, kcgs_2, kcgs_3, kcgs_4, kesg, sustin, moodys, msci, sp = row_data
        
            elif len(row_data) > 10: # 기업의 이름이 공백을 포함한 문자열인 경우 (ex. 'CJ' 'CGV'로 나뉘는 경우 'CJ CGV'로 합쳐야 한다)
                company_name = ' '.join(row_data[:len(row_data)-10+1])
                row_data = row_data[len(row_data)-10+1:] # 앞부분을 제외하여 슬라이싱
                kcgs_1, kcgs_2, kcgs_3, kcgs_4, kesg, sustin, moodys, msci, sp = row_data # 나머지 부분들 할당
                
            else: # 예상하지 못한 오류 상황
                company_name, kcgs_1, kcgs_2, kcgs_3, kcgs_4, kesg, sustin, moodys, msci, sp = ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-'] # 공백으로 처리
            
            # 기업별 추출한 정보를 dataframe의 한 행으로 저장
            data_df_2021.loc[len(data_df_2021)] = [company_name, kcgs_1, kcgs_2, kcgs_3, kcgs_4, kesg, moodys, msci, sp, year]

            # index를 증가시키기 -> while이 반복되며 index가 LIMIT이 되면 종료
            index += 1
            if index == LIMIT:
                break
        
        # for문을 다 돌아서 10개의 기업 정보를 크롤링하고 나면 다음 페이지를 클릭해야 한다
        next_page = driver.find_element(By.CSS_SELECTOR, "ul > li.next > a")
        next_page.click()
        time.sleep(5) # 다음 페이지가 로딩될 때까지 일정 시간 기다려서 오류를 방지
        
    # while문 종료
    data_df_2021.to_csv('esg_2021.csv')
    return data_df_2021

In [8]:
#2020년 esg등급 표 스크랩 함수
def esg_2020_scraper():
    driver = webdriver.Chrome()
    driver.get('https://esg.krx.co.kr/contents/02/02020000/ESG02020000.jsp')
    time.sleep(10)
    
    # 연도를 2022년으로 바꾸고 정보 조회
    select = Select(driver.find_element(By.CSS_SELECTOR, "#sch_yyc4ca4238a0b923820dcc509a6f75849b"))
    select.select_by_value('2020')
    driver.find_element(By.CSS_SELECTOR, "#btnidc4ca4238a0b923820dcc509a6f75849b").click()
    time.sleep(10)
    
    data_df_2020 = pd.DataFrame(columns=("회사명", "KCGS_종합등급", "KCGS_환경", "KCGS_사회", "KCGS_지배구조", "한국ESG연구소", "Moody's", "MSCI", "S&P", "연도"))
    
    year = 2020 # 매번 크롤링할 필요가 없으니 미리 선언하고 추가할 때 붙여주기만 하면 된다
    index = 0;  LIMIT = 794

    while index < LIMIT: # 총 기업이 794개가 있기 때문에 794로 제한을 둡니다.
        
        # 한 페이지 속 기업들(rows)에 대해(마지막 페이지를 제외하고 기본적으로 10개의 기업으로 구성)
        for i in range(1, 11):
            time.sleep(3)
            print(index + 1, end=' ') # 코드가 잘 돌아가는지 확인하기 위한 출력
    
            row = driver.find_element(By.CSS_SELECTOR, f"tbody > tr:nth-child({i})")

            row_data = row.text.replace('\n', ' ').split() # 빈칸을 기준으로 나누어 저장
            # 기업명, KCGS_종합 등급, KCGS_환경 등급, KCGS_사회 등급, KCGS_지배구조 등급, 한국ESG연구소 등급, 서스틴베스트 등급, Moody's 등급, MSCI 등급, S&P 등급
            print(row_data)
            if len(row_data) == 10: # 평범한 케이스
                company_name, kcgs_1, kcgs_2, kcgs_3, kcgs_4, kesg, sustin, moodys, msci, sp = row_data
        
            elif len(row_data) > 10: # 기업의 이름이 공백을 포함한 문자열인 경우 (ex. 'CJ' 'CGV'로 나뉘는 경우 'CJ CGV'로 합쳐야 한다)
                company_name = ' '.join(row_data[:len(row_data)-10+1])
                row_data = row_data[len(row_data)-10+1:] # 앞부분을 제외하여 슬라이싱
                kcgs_1, kcgs_2, kcgs_3, kcgs_4, kesg, sustin, moodys, msci, sp = row_data # 나머지 부분들 할당
                
            else: # 예상하지 못한 오류 상황
                company_name, kcgs_1, kcgs_2, kcgs_3, kcgs_4, kesg, sustin, moodys, msci, sp = ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-'] # 공백으로 처리
            
            # 기업별 추출한 정보를 dataframe의 한 행으로 저장
            data_df_2020.loc[len(data_df_2020)] = [company_name, kcgs_1, kcgs_2, kcgs_3, kcgs_4, kesg, moodys, msci, sp, year]

            # index를 증가시키기 -> while이 반복되며 index가 LIMIT이 되면 종료
            index += 1
            if index == LIMIT:
                break
        
        # for문을 다 돌아서 10개의 기업 정보를 크롤링하고 나면 다음 페이지를 클릭해야 한다
        next_page = driver.find_element(By.CSS_SELECTOR, "ul > li.next > a")
        next_page.click()
        time.sleep(5) # 다음 페이지가 로딩될 때까지 일정 시간 기다려서 오류를 방지
        
    # while문 종료
    data_df_2020.to_csv('esg_2020.csv')
    return data_df_2020

In [5]:
data_df_2023 = esg_2023_scraper()

1 ['AJ네트웍스', 'B+', 'C', 'B+', 'B+', '-', '-', '-', '-', '-']
2 ['AK홀딩스', 'B+', 'B', 'A', 'B', '-', '-', '-', '-', '4']
3 ['BGF', 'B+', 'A', 'A+', 'B', '-', '-', '-', '-', '18']
4 ['BGF리테일', 'A', 'A', 'A+', 'A', 'A', 'AA', '23', 'BBB', '58']
5 ['BNK금융지주', 'A', 'A', 'A', 'A', 'A', 'A', '37', 'A', '43']
6 ['BYC', 'D', 'D', 'D', 'D', '-', '-', '-', '-', '-']
7 ['CJ', 'B+', 'B+', 'A', 'B+', 'A', 'AA', '47', 'A', '29']
8 ['CJ', 'CGV', 'B+', 'B', 'B+', 'B+', 'B+', '-', '-', '-', '10']
9 ['CJ대한통운', 'B+', 'B+', 'A+', 'B', 'B+', 'BB', '39', 'BBB', '62']
10 ['CJ씨푸드', 'B', 'C', 'B', 'B', '-', '-', '-', '-', '-']
11 ['CJ제일제당', 'A', 'A', 'A', 'B+', 'A+', 'AA', '36', 'AA', '80']
12 ['CS홀딩스', 'C', 'D', 'D', 'B', '-', '-', '-', '-', '-']
13 ['DB', 'B', 'D', 'B', 'B+', '-', '-', '-', '-', '-']
14 ['DB금융투자', 'B', 'C', 'B', 'B', '-', '-', '-', '-', '4']
15 ['DB손해보험', 'B+', 'B+', 'A', 'B+', 'B+', 'AA', '33', 'A', '50']
16 ['DB하이텍', 'B', 'B', 'B+', 'B', 'A', 'AA', '-', '-', '20']
17 ['DGB금융지주', 'A', 'A+', '

,회사명,KCGS_종합등급,KCGS_환경,KCGS_사회,KCGS_지배구조,한국ESG연구소,Moody's,MSCI,S&P,연도
0,AJ네트웍스,B+,C,B+,B+,-,-,-,-,2023
1,AK홀딩스,B+,B,A,B,-,-,-,4,2023
2,BGF,B+,A,A+,B,-,-,-,18,2023
3,BGF리테일,A,A,A+,A,A,23,BBB,58,2023
4,BNK금융지주,A,A,A,A,A,37,A,43,2023
...,...,...,...,...,...,...,...,...,...,...
789,휴니드테크놀러지스,D,D,C,C,-,-,-,-,2023
790,휴비스,B+,B+,A,B+,-,-,-,6,2023
791,휴스틸,D,D,D,C,-,-,-,-,2023
792,흥국화재,B+,C,B+,B+,-,-,-,-,2023


In [9]:
data_df_2022 = esg_2022_scraper()

1 ['AJ네트웍스', 'B+', 'C', 'B+', 'B+', '-', '-', '-', '-', '-']
2 ['AK홀딩스', 'B+', 'B', 'A', 'B', '-', '-', '-', '-', '4']
3 ['BGF', 'B+', 'A', 'A+', 'B', '-', '-', '-', '-', '18']
4 ['BGF리테일', 'A', 'A', 'A+', 'A', 'A', '-', '23', 'BBB', '58']
5 ['BNK금융지주', 'A', 'A', 'A', 'A', 'A', '-', '37', 'A', '43']
6 ['BYC', 'D', 'D', 'D', 'D', '-', '-', '-', '-', '-']
7 ['CJ', 'B+', 'B+', 'A', 'B+', 'A', '-', '14', 'A', '29']
8 ['CJ', 'CGV', 'B+', 'B', 'B+', 'B+', 'B+', '-', '-', '-', '10']
9 ['CJ대한통운', 'B+', 'B+', 'A+', 'B', 'B+', '-', '39', 'BBB', '62']
10 ['CJ씨푸드', 'B', 'C', 'B', 'B', '-', '-', '-', '-', '-']
11 ['CJ제일제당', 'A', 'A', 'A', 'B+', 'S', '-', '36', 'AA', '80']
12 ['CS홀딩스', 'C', 'D', 'D', 'B', '-', '-', '-', '-', '-']
13 ['DB', 'B', 'D', 'B', 'B+', '-', '-', '-', '-', '-']
14 ['DB금융투자', 'B', 'C', 'B', 'B', '-', '-', '-', '-', '4']
15 ['DB손해보험', 'B+', 'B+', 'A', 'B+', 'B+', '-', '33', 'A', '50']
16 ['DB하이텍', 'B', 'B', 'B+', 'B', 'A', '-', '-', '-', '20']
17 ['DGB금융지주', 'A', 'A+', 'A+', 'A

In [10]:
data_df_2021 = esg_2021_scraper()

1 ['AJ네트웍스', 'B', 'D', 'B', 'B', '-', '-', '-', '-', '-']
2 ['AK홀딩스', 'B+', 'B', 'B+', 'B+', '-', '-', '-', '-', '1']
3 ['BGF', 'A', 'A', 'A+', 'A', '-', '-', '-', '-', '13']
4 ['BGF리테일', 'A', 'A', 'A+', 'A', 'S', '-', '-', 'BBB', '11']
5 ['BNK금융지주', 'A+', 'A', 'A+', 'A+', 'A', '-', '-', 'A', '38']
6 ['BYC', 'B', 'D', 'B', 'B+', '-', '-', '-', '-', '-']
7 ['CJ', 'A', 'B+', 'A+', 'A', 'A', '-', '-', 'BBB', '16']
8 ['CJ', 'CGV', 'A', 'B', 'A', 'A', 'B+', '-', '-', '-', '12']
9 ['CJ대한통운', 'A', 'A', 'A+', 'A', 'A', '-', '-', 'BBB', '62']
10 ['CJ씨푸드', 'B', 'C', 'B+', 'B+', '-', '-', '-', '-', '-']
11 ['CJ제일제당', 'A', 'A', 'A+', 'A', 'S', '-', '-', 'A', '78']
12 ['CS홀딩스', 'B', 'D', 'C', 'B+', '-', '-', '-', '-', '-']
13 ['DB', 'B', 'D', 'B+', 'B+', '-', '-', '-', '-', '-']
14 ['DB금융투자', 'B', 'D', 'B+', 'B', '-', '-', '-', '-', '-']
15 ['DB손해보험', 'A', 'A', 'A+', 'B+', 'A', '-', '-', 'BBB', '57']
16 ['DB하이텍', 'A', 'A', 'A', 'A', 'A', '-', '-', '-', '17']
17 ['DGB금융지주', 'A+', 'A', 'A+', 'A+', '-

In [12]:
data_df_2020 = esg_2020_scraper()

1 ['AJ네트웍스', 'B', 'D', 'B', 'B', '-', '-', '-', '-', '-']
2 ['AK홀딩스', 'B+', 'C', 'B+', 'B+', '-', '-', '-', '-', '-']
3 ['BGF', 'B+', 'B', 'A', 'B+', '-', '-', '-', '-', '-']
4 ['BGF리테일', 'A', 'B+', 'B+', 'A', '-', '-', '-', 'BB', '-']
5 ['BNK금융지주', 'A+', 'B+', 'A+', 'A+', '-', '-', '-', 'A', '-']
6 ['BYC', 'B', 'D', 'B', 'B+', '-', '-', '-', '-', '-']
7 ['CJ', 'A', 'B', 'A', 'A', '-', '-', '-', 'BBB', '-']
8 ['CJ', 'CGV', 'A', 'B', 'A', 'A', '-', '-', '-', '-', '-']
9 ['CJ대한통운', 'A', 'A', 'B', 'A', '-', '-', '-', 'B', '-']
10 ['CJ씨푸드', 'B', 'C', 'B+', 'B+', '-', '-', '-', '-', '-']
11 ['CJ제일제당', 'A', 'A', 'A+', 'A', '-', '-', '-', 'A', '-']
12 ['CS홀딩스', 'C', 'D', 'C', 'B', '-', '-', '-', '-', '-']
13 ['DB', 'B', 'D', 'B', 'B', '-', '-', '-', '-', '-']
14 ['DB금융투자', 'B', 'D', 'A', 'B', '-', '-', '-', '-', '-']
15 ['DB손해보험', 'B+', 'A', 'A', 'B+', '-', '-', '-', 'BBB', '-']
16 ['DB하이텍', 'B+', 'B+', 'A', 'B+', '-', '-', '-', '-', '-']
17 ['DGB금융지주', 'A+', 'A', 'A+', 'A+', '-', '-', '-', '

In [13]:
# 데이터 합치기
data_df_merge = pd.concat([data_df_2023, data_df_2022, data_df_2021, data_df_2020])
data_df_merge

,Unnamed: 0,회사명,KCGS_종합등급,KCGS_환경,KCGS_사회,KCGS_지배구조,한국ESG연구소,Moody's,MSCI,S&P,연도
0,0.0,AJ네트웍스,B+,C,B+,B+,-,-,-,-,2023
1,1.0,AK홀딩스,B+,B,A,B,-,-,-,4,2023
2,2.0,BGF,B+,A,A+,B,-,-,-,18,2023
3,3.0,BGF리테일,A,A,A+,A,A,23,BBB,58,2023
4,4.0,BNK금융지주,A,A,A,A,A,37,A,43,2023
...,...,...,...,...,...,...,...,...,...,...,...
789,NaN,휴니드테크놀러지스,C,C,B,B,-,-,-,-,2020
790,NaN,황금에스티,D,D,C,C,-,-,-,-,2020
791,NaN,휴스틸,C,B,B,C,-,-,-,-,2020
792,NaN,흥국화재,B,D,A,B,-,-,-,-,2020


In [14]:
data_df_merge = data_df_merge.drop('Unnamed: 0', axis=1)

In [15]:
data_df_merge

,회사명,KCGS_종합등급,KCGS_환경,KCGS_사회,KCGS_지배구조,한국ESG연구소,Moody's,MSCI,S&P,연도
0,AJ네트웍스,B+,C,B+,B+,-,-,-,-,2023
1,AK홀딩스,B+,B,A,B,-,-,-,4,2023
2,BGF,B+,A,A+,B,-,-,-,18,2023
3,BGF리테일,A,A,A+,A,A,23,BBB,58,2023
4,BNK금융지주,A,A,A,A,A,37,A,43,2023
...,...,...,...,...,...,...,...,...,...,...
789,휴니드테크놀러지스,C,C,B,B,-,-,-,-,2020
790,황금에스티,D,D,C,C,-,-,-,-,2020
791,휴스틸,C,B,B,C,-,-,-,-,2020
792,흥국화재,B,D,A,B,-,-,-,-,2020


In [16]:
# 정렬하기
data_df_merge.sort_values(by=['회사명', '연도'], inplace = True)
data_df_merge

,회사명,KCGS_종합등급,KCGS_환경,KCGS_사회,KCGS_지배구조,한국ESG연구소,Moody's,MSCI,S&P,연도
0,AJ네트웍스,B,D,B,B,-,-,-,-,2020
0,AJ네트웍스,B,D,B,B,-,-,-,-,2021
0,AJ네트웍스,B+,C,B+,B+,-,-,-,-,2022
0,AJ네트웍스,B+,C,B+,B+,-,-,-,-,2023
1,AK홀딩스,B+,C,B+,B+,-,-,-,-,2020
...,...,...,...,...,...,...,...,...,...,...
792,흥국화재,B+,C,B+,B+,-,-,-,-,2023
793,흥아해운,D,D,C,D,-,-,-,-,2020
793,흥아해운,D,D,C,D,-,-,-,-,2021
793,흥아해운,C,C,B,C,-,-,-,-,2022


In [20]:
data_df_merge.reset_index(drop=True, inplace=True)

In [21]:
data_df_merge

,회사명,KCGS_종합등급,KCGS_환경,KCGS_사회,KCGS_지배구조,한국ESG연구소,Moody's,MSCI,S&P,연도
0,AJ네트웍스,B,D,B,B,-,-,-,-,2020
1,AJ네트웍스,B,D,B,B,-,-,-,-,2021
2,AJ네트웍스,B+,C,B+,B+,-,-,-,-,2022
3,AJ네트웍스,B+,C,B+,B+,-,-,-,-,2023
4,AK홀딩스,B+,C,B+,B+,-,-,-,-,2020
...,...,...,...,...,...,...,...,...,...,...
3171,흥국화재,B+,C,B+,B+,-,-,-,-,2023
3172,흥아해운,D,D,C,D,-,-,-,-,2020
3173,흥아해운,D,D,C,D,-,-,-,-,2021
3174,흥아해운,C,C,B,C,-,-,-,-,2022


In [22]:
# 저장하기
data_df_merge.to_csv('esg_merge.csv', encoding='utf-8-sig')